In [1]:
import spacy
nlp = spacy.load('en')

In [52]:
s1 = 'The Searching is directed by Aneesh in his feature debut and written by Chaganty, Ohanian and John in 2018.'
s2 = 'The Mission Impossible was written by Christopher Nolan.'
s3 = 'The Searching is directed by Aneesh in his feature debut and written by Chaganty in 2018.'
s4 = 'herry film was written and produced by Joss Whedon and directed by Jorge and Josh.'
s6 = 'Chrisopher wrote Mission Impossible.'
s7 = 'Mission: Impossible – Rogue Nation was written and directed by Christopher McQuarrie'
s8 = 'Black Panther was co-written by Ryan Coogler, Joe Robert.'
# s8 is not going to be detected as there is '-' between 'co' and 'written'
s5 = 'herry film was written and produced by Joss Whedon and Jorge.'
s9 = 'The Jury was written and co-produced by Rose and directed by Sidney Lumet.'
#s9 won't work
s10 = 'The Jury was written and produced by Rose and directed by Sidney Lumet.'

In [55]:
sentence = s10
parse(sentence)

Matching with wriiten template: for The Jury was written and produced by Rose and directed by Sidney Lumet.
Has valid verb:True
--------- Extracted Information -----------
Movie name : The Jury
Writer name :  ['Rose']
Time :  <no-match>


In [54]:
def parse(statement1):
    isValidForWrittenTemplate = checkWrittenTemplateEligibility(statement1)
    if(isValidForWrittenTemplate):

        print("--------- Extracted Information -----------")
        
        movieName = extractMovieName(statement1)
        print("Movie name : " + str(movieName))

        writerName = extractWriterName(statement1)
        print("Writer name : " , writerName)

        year = extractYear(statement1)
        print("Time : " , year)
    else:
        print(" NO - PARSE ")

In [119]:
# statement1 = 'The Mission Impossible was written by Christopher Nolan.'
statement1 = 'The Searching is directed by Aneesh in his feature debut and written by Chaganty, Ohanian and John in 2018.'
# statement1 = 'herry film was written and produced by Joss Whedon and directed by Jorge and Josh.'
#statement1 = 'herry film was written and produced by Joss Whedon and Jorge.'
statement2 = 'Chrisopher Nolan wrote Mission Impossible.'


isValidForWrittenTemplate = checkWrittenTemplateEligibility(statement1)
if(isValidForWrittenTemplate):
    
    movieName = extractMovieName(statement1)
    print("extracted movie name is:" + str(movieName))
    
    writerName = extractWriterName(statement1)
    print("extracted writer name is:" , writerName)
    
    year = extractYear(statement1)
    print("extracted year is:" , year)


Matching with wriiten template: for The Searching is directed by Aneesh in his feature debut and written by Chaganty, Ohanian and John in 2018.
Has valid verb:True
vt:written
extracted movie name is:The Searching
extracted writer name is: ['Chaganty', 'John', 'Ohanian']
extracted year is: 2018


In [2]:
def extractMovieName(statement):
    targetVerbLemmas = ['write']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    movieName = "<uninitialized>"
    
#     handle when release is with premiered.
    '''
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to : "+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    '''

    #print("vt:"+verbText)

    movieName = getMovie(statement,verbText)
    
    return movieName

In [3]:
def extractWriterName(statement):
    targetVerbLemmas = ['write']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    writerName = "<uninitialized>"
    
#     handle when release is with premiered.
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        #print("verbTextChanged to"+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)

    #print("vt:"+verbText)

    writerName = getWriter(statement,verbText)
    
    return writerName

In [4]:
def extractYear(statement):
    doc = nlp(statement)
    for ent in doc.ents:
        if (ent.label_ == "DATE" or ent.label_ == "TIME"):
            return ent.text
    
    return "<no-match>"

In [5]:
def getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas):
    """
    this will reutrn exact apperance of verb in statement.
    ex: statement = "Movie was written by James.", targetVerbLemmas = ['write','script']
    this will return "written" as it's lemma matches with one of the targetVerbLemmas
    """
    verbTokens = filterVerbTokens(statement)
    for token in verbTokens:
        if token.lemma_ in targetVerbLemmas:
            return token.text
    return "<no-match>"

In [6]:
def getNounSubject(statement, verbText):
    doc = nlp(statement)
    for token in doc:
        print(token.text, token.dep_, token.head.text)
        if((token.dep_ == "nsubjpass" or token.dep_ == "nsubj") and token.head.text == verbText):
            return token.text
    return "<no-match>"

In [7]:
def getMovie(statement, verbText):
    doc = nlp(statement)
    theme = "<uninitialized>" #movie
    
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "nsubjpass" and token.head.text == verbText):
            #print("token nsubjpass : " + token.text)
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif (token.dep_ == "dobj" and token.head.text == verbText):
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif ((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            #print("token conj : " + token.text)
            if (token.head.dep_ == "ROOT"):
                #print("token.head: " + token.head.text)
                token1 = token.head
                return getMovie(statement, token1.text)
                '''
                if (token.dep_ == "nsubjpass" and token.head.text == token1.text):
                    theme = getNounChunkThatContainsNoun(statement, token)
                    return theme
                '''
    
    return "<no-match>"

In [8]:
def getWriter(statement, verbText):
    doc = nlp(statement)
    agent = "<uninitialized>" #writer
    '''
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        elif (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
    '''
    
    for token in doc:
        #print("token:",token)
        '''
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            print("Agent (agent) : " + agent)
            return agent
        '''
        if (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            #print("Agent (nsubj) : " + agent)
            return agent
        
        elif((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            agent = extractOwnerEntitySyntactically(statement)
            return agent
        
        elif(token.head.text == verbText or token.text == verbText):
            agent = extractOwnerEntitySyntactically(statement)
            return agent
        
    return agent

In [9]:
def extractOwnerEntitySyntactically(statement):
    doc = nlp(statement)
    verbTokens = filterVerbTokens(statement)
    ORGEntity = []
    
    for token in verbTokens:
        if isExpectedVerbToken(token):
            #print("Expected token is:"+token.text)
            #expected children [was, theatrically, in, on, by]
            byToken = None
            for child in token.children: 
                #usually release location starts with "in"
                if child.text == "by":
                    byToken = child
             
            if byToken == None:
                #means no direct by available so lets look at some conj child
                for child in token.children: 
                    #usually release location starts with "in"
                    if child.dep_ == "conj":
                        for grandChild in child.children:
                            if grandChild.text == "by":
                                byToken = grandChild
            
            if byToken != None:
                for subChild in byToken.children:
                    ORGEntity.append(getORGorPersonThatContainsToken(statement, subChild))
                    #print("subchild : " + subChild.text)
                    if (subChild.dep_ == "pobj" and subChild.head.text == "by"):
                        grandChildren = subChild.children
                        childEntity = getORGEntityForGrandChild1(statement, grandChildren, ORGEntity)
                        #print("child entity:    " , str(childEntity))
                        return list(set(childEntity))
                        #ORGEntity.clear()
                        #for item in childEntity:
                            #ORGEntity.append(item)
                    return ORGEntity
    return "<no-match>"

In [10]:
def getORGEntityForGrandChild1(statement, children, ORGEntity):
    '''
    New definition
    '''
    doc = nlp(statement)
    
    for child in children:
        sizeChild = sum(1 for item in child.children)
        #print(child, sizeChild)
        if sizeChild > 0:
            ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
            #print(ORGEntity)
            #ORGEntity.append(getORGEntityForGrandChild1(statement, child.children, ORGEntity))
            getORGEntityForGrandChild1(statement, child.children, ORGEntity)
        if sizeChild == 0:
            if (getORGorPersonThatContainsToken(statement, child) != "<no-match>"):
                ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
    
    return ORGEntity

In [11]:
def isExpectedVerbToken(token):
    return token.lemma_ in ["write"]

In [12]:
def getORGorPersonThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return "<no-match>"

In [13]:
import re
def calculateStartPositionOfToken(statement, targetToken):
    """
    this should return start position of the token
    """
    matches = [m.start() for m in re.finditer(targetToken.text, statement)]
    if(len(matches)==1):
        return matches[0]
    

    huristicPosition = 0
    doc = nlp(statement)
    
    i = 0
    for token in doc:
        if i < targetToken.i:
            huristicPosition = huristicPosition + len(token.text)+1
            i = i+1
        else:
            break
        
    distances = []
    for match in matches:
        distances.append(abs(match-huristicPosition))
        
    minIndex = distances.index(min(distances))
    
    return matches[minIndex]


In [14]:
def getNounChunkThatContainsNoun(statement, nounSubject):
    doc = nlp(statement)
    for chunk in doc.noun_chunks:
        if nounSubject.text in chunk.text:
            return chunk.text
    return "<no-match>"

In [15]:
def checkWrittenTemplateEligibility(statement):
    print("Matching with wriiten template: for " + statement)
    hasValidVerb = check_verb_match(statement)
    print("Has valid verb:" + str(hasValidVerb))
    
    return hasValidVerb

In [16]:
def check_verb_match(statement):
    verbTokens = filterVerbTokens(statement)
    targetVerbs = ['write']
    for token in verbTokens:
        if token.lemma_ in targetVerbs:
            return True
    return False

In [17]:
def filterVerbTokens(statement):
    doc = nlp(statement)
    result = list(filter(lambda token: token.pos_ == "VERB" and token.lemma_ != "be", doc))
    return result

In [18]:
def get_dependency_tree_nodes(sentence):
    nodes = []
    doc = nlp(sentence)
    for token in doc:
        nodes.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
    return nodes

In [19]:
get_dependency_tree_nodes('The Mission Impossible was written by Chrisopher Nolan.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'written', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'written', 'VERB', []],
 ['written', 'ROOT', 'written', 'VERB', [Impossible, was, by, .]],
 ['by', 'agent', 'written', 'VERB', [Nolan]],
 ['Chrisopher', 'compound', 'Nolan', 'PROPN', []],
 ['Nolan', 'pobj', 'by', 'ADP', [Chrisopher]],
 ['.', 'punct', 'written', 'VERB', []]]

In [20]:
get_dependency_tree_nodes('Chrisopher Nolan wrote Mission Impossible.')

[['Chrisopher', 'compound', 'Nolan', 'PROPN', []],
 ['Nolan', 'nsubj', 'wrote', 'VERB', [Chrisopher]],
 ['wrote', 'ROOT', 'wrote', 'VERB', [Nolan, Impossible, .]],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'dobj', 'wrote', 'VERB', [Mission]],
 ['.', 'punct', 'wrote', 'VERB', []]]

In [21]:
get_dependency_tree_nodes('The Searching is directed by Aneesh in his feature debut and written by Chaganty, Ohanian and John.')

[['The', 'det', 'Searching', 'PROPN', []],
 ['Searching', 'nsubjpass', 'directed', 'VERB', [The]],
 ['is', 'auxpass', 'directed', 'VERB', []],
 ['directed',
  'ROOT',
  'directed',
  'VERB',
  [Searching, is, by, in, and, written, .]],
 ['by', 'agent', 'directed', 'VERB', [Aneesh]],
 ['Aneesh', 'pobj', 'by', 'ADP', []],
 ['in', 'prep', 'directed', 'VERB', [debut]],
 ['his', 'poss', 'debut', 'NOUN', []],
 ['feature', 'compound', 'debut', 'NOUN', []],
 ['debut', 'pobj', 'in', 'ADP', [his, feature]],
 ['and', 'cc', 'directed', 'VERB', []],
 ['written', 'conj', 'directed', 'VERB', [by]],
 ['by', 'agent', 'written', 'VERB', [Chaganty]],
 ['Chaganty', 'pobj', 'by', 'ADP', [,, Ohanian]],
 [',', 'punct', 'Chaganty', 'PROPN', []],
 ['Ohanian', 'conj', 'Chaganty', 'PROPN', [and, John]],
 ['and', 'cc', 'Ohanian', 'PROPN', []],
 ['John', 'conj', 'Ohanian', 'PROPN', []],
 ['.', 'punct', 'directed', 'VERB', []]]

In [22]:
get_dependency_tree_nodes('herry film was written and directed by Joss Whedon and Jorge.')

[['herry', 'amod', 'film', 'NOUN', []],
 ['film', 'nsubjpass', 'written', 'VERB', [herry]],
 ['was', 'auxpass', 'written', 'VERB', []],
 ['written', 'ROOT', 'written', 'VERB', [film, was, and, directed, .]],
 ['and', 'cc', 'written', 'VERB', []],
 ['directed', 'conj', 'written', 'VERB', [by]],
 ['by', 'agent', 'directed', 'VERB', [Whedon]],
 ['Joss', 'compound', 'Whedon', 'PROPN', []],
 ['Whedon', 'pobj', 'by', 'ADP', [Joss, and, Jorge]],
 ['and', 'cc', 'Whedon', 'PROPN', []],
 ['Jorge', 'conj', 'Whedon', 'PROPN', []],
 ['.', 'punct', 'written', 'VERB', []]]

In [23]:
get_dependency_tree_nodes('herry film was written and produced by Joss Whedon and directed by jorge.')

[['herry', 'amod', 'film', 'NOUN', []],
 ['film', 'nsubjpass', 'written', 'VERB', [herry]],
 ['was', 'auxpass', 'written', 'VERB', []],
 ['written', 'ROOT', 'written', 'VERB', [film, was, and, produced, .]],
 ['and', 'cc', 'written', 'VERB', []],
 ['produced', 'conj', 'written', 'VERB', [by, and, directed]],
 ['by', 'agent', 'produced', 'VERB', [Whedon]],
 ['Joss', 'compound', 'Whedon', 'PROPN', []],
 ['Whedon', 'pobj', 'by', 'ADP', [Joss]],
 ['and', 'cc', 'produced', 'VERB', []],
 ['directed', 'conj', 'produced', 'VERB', [by]],
 ['by', 'agent', 'directed', 'VERB', [jorge]],
 ['jorge', 'pobj', 'by', 'ADP', []],
 ['.', 'punct', 'written', 'VERB', []]]

In [24]:
doc = nlp("Dark Knight is directed by Aneesh in his feature debut and written by Chaganty, Sev Ohanian and John.")
for chunk in doc.noun_chunks:
    print(chunk)

Dark Knight
Aneesh
his feature debut
Chaganty
Sev Ohanian
John
